In [1]:
import openvsp as vsp
from import_prop import process_propeller_file 
import os

In [2]:
vsp.VSPCheckSetup()
vsp.VSPRenew()
vsp.ClearVSPModel()
vsp.DeleteAllResults()


# Import the propeller geometry
prop_id = vsp.ImportFile('geometry/10x10E-PERF.bem', vsp.IMPORT_BEM, "")

vsp.SetParmVal(prop_id, "ConstructXoC", "Design", 0.0)  # Construct x/c at 0.0
vsp.SetParmVal(prop_id, "FeatherAxisXoC", "Design", 0.125)  # Set the feather axis to 0.125

# Set up root airfoil section
xsec_surf = vsp.GetXSecSurf(prop_id, 0)
vsp.ChangeXSecShape(xsec_surf, 0, vsp.XS_FILE_AIRFOIL)

xsec = vsp.GetXSec(xsec_surf, 0)
vsp.ReadFileAirfoil(xsec, "./geometry/e63.dat")

# Set up tip airfoil section
xsec_surf = vsp.GetXSecSurf(prop_id, 1)
vsp.ChangeXSecShape(xsec_surf, 1, vsp.XS_FILE_AIRFOIL)

xsec = vsp.GetXSec(xsec_surf, 1)
vsp.ReadFileAirfoil(xsec, "./geometry/n4412.dat")

# Set tesselation
vsp.SetParmVal(prop_id, "Tess_U", "Shape", 32)  # Num_U 
vsp.SetParmVal(prop_id, "Tess_W", "Shape", 32)  # Num_W

vsp.SetParmVal(vsp.FindParm(vsp.FindUnsteadyGroup(0),"RPM","UnsteadyGroup"), 2500)

vsp.Update()
vsp.WriteVSPFile("apc_bem_10x10.vsp3")

In [3]:
# Comp Geometry
analysis_name = "VSPAEROComputeGeometry"
vsp.SetAnalysisInputDefaults(analysis_name)
analysis_method = list(vsp.GetIntAnalysisInput(analysis_name, "AnalysisMethod" ))
analysis_method[0] = vsp.VORTEX_LATTICE
vsp.SetIntAnalysisInput(analysis_name, "AnalysisMethod", analysis_method)
geom_res_id = vsp.ExecAnalysis( analysis_name )


In [4]:

# Analysis Setup
analysis_name="VSPAEROSweep"
vsp.SetAnalysisInputDefaults("VSPAEROSweep")
aero_id = vsp.FindContainer( "VSPAEROSettings", 0)


# recref=[50000]
# recref_end=[50000]
vref=[20]
blades_flag=[1]
vinf=[20]
rho=[1.225]
alpha_start=[0]
alpha_end=[0]
auto_time_flag=[1]
num_rev_flag=[5]
# numtimesteps=[120]
# timestepsizes=[0.00042]

# vsp.SetDoubleAnalysisInput(analysis_name,"ReCref",recref,0)
# vsp.SetDoubleAnalysisInput(analysis_name,"ReCrefEnd",recref_end,0)
vsp.SetDoubleAnalysisInput(analysis_name,"Vref",vref, 0)
vsp.SetIntAnalysisInput(analysis_name,"RotateBladesFlag",blades_flag,0)
vsp.SetDoubleAnalysisInput(analysis_name,"Vinf",vinf,0)
vsp.SetDoubleAnalysisInput(analysis_name,"Rho",rho,0)
vsp.SetDoubleAnalysisInput(analysis_name,"AlphaStart", alpha_start,0)
vsp.SetIntAnalysisInput(analysis_name,"AutoTimeStepFlag",auto_time_flag,0)
vsp.SetIntAnalysisInput(analysis_name,"AutoTimeNumRevs", num_rev_flag,0)
# vsp.SetDoubleAnalysisInput(analysis_name,"NumTimeSteps",numtimesteps,0)
# vsp.SetDoubleAnalysisInput(analysis_name,"TimeStepSize",timestepsizes,0)


vsp.Update()
vsp.PrintAnalysisInputs(analysis_name)
vsp.ExecAnalysis(analysis=analysis_name)

'LMKLSWF'

In [6]:
rotor_res = vsp.FindResultsID("VSPAERO_Rotor", 0)
vsp.WriteResultsCSVFile(rotor_res, "apc_bem_10x10.csv")